<a href="https://colab.research.google.com/github/xhaktmchl/SAI_2020-2_keras_study/blob/main/SAI_keras_5_1%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#5장 컴퓨터 비전을 위한 딥러닝

#5.1 합성곱 신경망 소개


In [ ]:
import keras
keras.__version__


'2.4.3'

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

이코드에서 주목할 점  
* Conv2D, MaxPooling2D 층 사용
* mnist 이미지 포맷이(28,28,1) 크기로 input_shape() 사용

모델의 모양 살펴보기

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Conv2D와 MaxPooling2D 층의 출력은 (height, width, channels) 크기의 3D 텐서인데  
마지막 층의 ((3, 3, 64) 크기인) 출력 텐서를 Dense층에 넣어야 한다  



* 요약 : 3차원 텐서 -> 1차원 텐서로 변환해야함  
Flatten() 층으로 변환

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

주목할 점: (3, 3, 64)의 3차원 텐서가 (576) 1차원 텐서로 변환됨  
마지막 10개의 클래스로 분류

mnist 예제로 컨브넷 훈련

In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 7s 8ms/step - loss: 0.1735 - accuracy: 0.9457
Epoch 2/5
938/938 [==============================] - 6s 7ms/step - loss: 0.0482 - accuracy: 0.9854
Epoch 3/5
938/938 [==============================] - 7s 7ms/step - loss: 0.0327 - accuracy: 0.9901
Epoch 4/5
938/938 [==============================] - 7s 7ms/step - loss: 0.0244 - accuracy: 0.9922
Epoch 5/5
938/938 [==============================] - 7s 7ms/step - loss: 0.0187 - accuracy: 0.9943


테스트셋으로 모델 평가

In [ ]:

test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 4ms/step - loss: 0.0296 - accuracy: 0.9907


In [ ]:
test_acc

0.9907000064849854

* 정확도 결과: 완전연결 모델(97%) < 컨브넷 모델(99%) 

### 5.1.1 합성곱 연산


* Dense층은 전역패턴 학습 != 합성곱(지역패턴 학습)
* 합성곱 특징 :  
 - 평행이동 불변성   
 : 학습된 패턴은 다른 지역에서도 인식 가능
 - 패턴의 공간적 계층구조 학습  
 : 앞의 합성곱 층의 패턴으로 더 큰 패턴을 학습 

* 단어 정의
 * 특성 맵: 깊이 축에 있는 각 차원은 하나의 특성 ==> (필터) 
 * 응답 맵 : 입력의 위치에서 각 패턴(특성맵)에 대한 2D 맵
 
* 합성곱 과정  
: 특성맵 입력 -> (3* 3또는5 * 5) 정도의 윈도우만큼의 크기만큼 검사  ->출력깊이 크기의 1차원 벡터로 변환 -> 1차원 벡터들 모두 합하면
(높이,너비,출력 깊이) 크기의 3차원 맵으로 재구성 


* 경계문제와 패딩  
 1) 경계문제  
 : 윈도우의 크기에 따라 입력 특성맵과 응답맵의 타일의 개수가 차이가 남  
 2) 패딩  
 : 타일의 개수가 동일하게 맞추도록 입력특성맵에 행,열 추가  
 예) padding 매개변수로 사용 (valid : 패딩사용 안함, same : 입력특성맵과 출력맵의 크기를 맞춘다)

* 스트라이드 합성곱
: 윈도우가 지나가는 간격 => 타일의 수가 줄어듬(다운 샘플링)



### 5.1.2최대 풀링 연산

* 최대풀링 연산(Maxpooling2D) 이란?  
: 강제적으로 특성맵을 다운 샘플링

* 다운 샘플링의 이유?
 - 처리할 특성맵의 가중치 개수 줄이기 -> 과대적합 방지
 - 원본 입력 면에서 윈도우가 적용되는 범위가  점점 커짐 -> 공간적 계층 학습 가능